In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [2]:
import pprint

from dotenv import load_dotenv
from IPython.display import display

from uniflow.flow.client import RaterClient
from uniflow.flow.flow_factory import FlowFactory
from uniflow.flow.config  import RaterClassificationConfig
from uniflow.op.model.model_config import BedrockModelConfig
from uniflow.viz import Viz
from uniflow.op.prompt_schema import Context

load_dotenv()

/home/ubuntu/anaconda3/envs/file_extraction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
FlowFactory.list()

{'extract': ['ExtractIpynbFlow',
  'ExtractMarkdownFlow',
  'ExtractPDFFlow',
  'ExtractTxtFlow'],
 'transform': ['TransformCopyFlow',
  'TransformHuggingFaceFlow',
  'TransformLMQGFlow',
  'TransformOpenAIFlow'],
 'rater': ['RaterFlow']}

In [9]:
raw_input = [
    ("The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south.",
     "What is the largest ocean on Earth?",
     "The largest ocean on Earth is the Pacific Ocean."), # correct
    ("Shakespeare, a renowned English playwright and poet, wrote 39 plays during his lifetime. His works include famous plays like 'Hamlet' and 'Romeo and Juliet'.",
     "How many plays did Shakespeare write?",
     "Shakespeare wrote 39 plays."), # correct
    ("The human brain is an intricate organ responsible for intelligence, memory, and emotions. It is made up of approximately 86 billion neurons.",
     "What is the human brain responsible for?",
     "The human brain is responsible for physical movement."), # incorrect
]

In [10]:
data = [
    Context(context=c[0], question=c[1], answer=c[2])
    for c in raw_input
]

In [6]:
config = RaterClassificationConfig(
    flow_name="RaterFlow",
    model_config=BedrockModelConfig(aws_profile="default", aws_region="us-west-2", model_kwargs={'temperature': 0.1}),
    label2score={"Yes": 1.0, "No": 0.0})
client = RaterClient(config)

RaterConfig(flow_name='RaterFlow', model_config={'model_name': 'anthropic.claude-v2', 'model_server': 'BedrockModelServer', 'batch_size': 1, 'aws_profile': 'default', 'aws_region': 'us-west-2', 'model_kwargs': {'temperature': 0.1}}, label2score={'Yes': 1.0, 'No': 0.0}, guided_prompt_template=GuidedPrompt(instruction='Rate the answer based on the question and the context.\n        Follow the format of the examples below to include context, question, answer, and label in the response.\n        The response should not include examples in the prompt.', examples=[Context(context='The Eiffel Tower, located in Paris, France, is one of the most famous landmarks in the world. It was constructed in 1889 and stands at a height of 324 meters.', question='When was the Eiffel Tower constructed?', answer='The Eiffel Tower was constructed in 1889.', explanation='The context explicitly mentions that the Eiffel Tower was constructed in 1889, so the answer is correct.', label='Yes'), Context(context='Pho

In [11]:
output = client.run(data)
pprint.pprint(output)

100%|██████████| 3/3 [00:08<00:00,  2.99s/it]

[{'output': [{'average_score': 1.0,
              'error': 'No errors.',
              'majority_vote': 'yes',
              'response': [' context: The Eiffel Tower, located in Paris, '
                           'France, is one of the most famous landmarks in the '
                           'world. It was constructed in 1889 and stands at a '
                           'height of 324 meters.\n'
                           'question: When was the Eiffel Tower constructed?\n'
                           'answer: The Eiffel Tower was constructed in 1889.\n'
                           'label: Yes\n'
                           '\n'
                           'context: Photosynthesis is a process used by '
                           'plants to convert light energy into chemical '
                           'energy. This process primarily occurs in the '
                           'chloroplasts of plant cells.  \n'
                           'question: Where does photosynthesis primarily '
 

In [12]:
for idx, o in enumerate(output):
    majority_vote = o['output'][0]['majority_vote']
    average_score = o['output'][0]['average_score']
    print(f"data {idx} has majority vote {majority_vote} and average score {average_score}")

data 0 has majority vote yes and average score 1.0
data 1 has majority vote yes and average score 1.0
data 2 has majority vote no and average score 0.0
